<h1 style="color:gray;">Import Packages and Functions</h1>

In [1]:
#Import Basic Pakages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#import pyodbc from db interaction
import pyodbc

#import excel package
import openpyxl

#For Web Scrapping
from bs4 import BeautifulSoup

#Import Requests - make HTTP requests in Python
import requests
from requests.exceptions import HTTPError

#import openpyxl in case excel operations is necessary
import openpyxl

#for wait for next execution
import time
import datetime

#Import IPython
from IPython.display import Javascript

<h1 style="color:gray;">Code</h1>

In [2]:
#Change Headers Request
headers =requests.utils.default_headers()

#Update user agent
headers.update({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'})

In [3]:
try:
    data = requests.get('http://tvefamosos.uol.com.br/bbb')

    # If successful, no problems
    data.raise_for_status()
except HTTPError as HTTPer:
    print(f'HTTP error ocurred: {HTTPer}')
except Exception as e:
    print(f'Other error ocurred: {e}')
else:
    pass

In [4]:
data.headers

{'Content-Type': 'text/html;charset=UTF-8', 'Content-Length': '34858', 'Connection': 'keep-alive', 'Date': 'Sat, 20 Feb 2021 16:26:28 GMT', 'Server': 'marrakesh 1.20.0', 'Cache-Control': 'no-transform, max-age=60, must-revalidate, proxy-revalidate', 'Last-Modified': 'Sat, 20 Feb 2021 16:24:46 GMT', 'Expires': 'Sat, 20 Feb 2021 16:27:28 GMT', 'Content-Encoding': 'gzip', 'Content-Security-Policy': "frame-ancestors 'self' http://*.intranet http://*.uolinc.com https://*.intranet https://*.uolinc.com;", 'ETag': '"a56d57461358d1f557cf1e22806f03f1"', 'Vary': 'Accept-Encoding,User-Agent', 'X-Cache': 'Hit from cloudfront', 'Via': '1.1 ac4e6581409ee543d30418bc80eff8a8.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'GRU50-C1', 'X-Amz-Cf-Id': '8LRO_hUdKlFoLFDDT5SordvOm_g6HnF8lQWzbdEjiy1DiVmgyqsejQ==', 'Age': '8'}

In [5]:
#classes of interest
caption = 'thumb-caption col-xs-5 col-sm-18 no-gutter'
img = 'thumbnails-item align-horizontal list col-xs-8 col-sm-12 small col-sm-24 small'
link = 'thumbnails-item align-horizontal list col-xs-8 col-sm-12 small col-sm-24 small'

In [6]:
info = BeautifulSoup(data.text,'html.parser')

In [7]:
print(info.prettify())

<!DOCTYPE html>
<html lang="pt-br">
 <head>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Especial BBB - UOL TV e Famosos - UOL TV e Famosos
  </title>
  <link crossorigin="anonymous" href="https://stc.uol.com" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://c.jsuol.com.br" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://conteudo.jsuol.com.br" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://conteudo.imguol.com.br" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://me.jsuol.com.br" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://www.google-analytics.com" rel="preconnect"/>
  <link href="https://stc.uol.com" rel="dns-prefetch"/>
  <link href="https://c.jsuol.com.br" rel="dns-prefetch"/>
  <link href="https://conteudo.jsuol.com.br" rel="dns-prefetch"/>
  <link href="https://conteudo.imguol.com.br" rel="dns-prefetch"/>
  <link href="https://me.js

In [8]:
# Connect to local DB
db_statement = r'Driver=ODBC Driver 17 for SQL Server;Server=your_server;Database=local_project;Trusted_Connection=yes;'

In [9]:
# query to add data to local db
sql = '''
Insert into news 
(caption, image, link,dt,dt_time)
values (?,?,?,?,?);
'''

In [ ]:
#loop forever
while True:
    if data.status_code:

        try:
            data = requests.get('http://tvefamosos.uol.com.br/bbb')

            # If successful, no problems
            data.raise_for_status()
        except HTTPError as HTTPer:
            print(f'HTTP error ocurred: {HTTPer}')
        except Exception as e:
            print(f'Other error ocurred: {e}')
        else:
            pass

        info = BeautifulSoup(data.text,'html.parser')

        caption_list = []
        image_list=[]
        date_list = []
        time_list =[]
        link_list=[]

        for tx in info.find_all('div',class_='thumb-caption col-xs-5 col-sm-18 no-gutter'):
            caption_list.append(tx.text)
            date_list.append(tx.text[-17:].strip().split(" ")[0])
            time_list.append(tx.text[-17:].strip().split(" ")[1])

        for i in info.find_all('div',class_='thumbnails-item align-horizontal list col-xs-8 col-sm-12 small col-sm-24 small'):
            image_list.append(i.figure.img['data-src'])

        for link in info.find_all('div',class_='thumbnails-item align-horizontal list col-xs-8 col-sm-12 small col-sm-24 small'):
            link_list.append(link.a['href'])
        

        df = pd.DataFrame(
            {
                "caption":caption_list,
                "image":image_list,
                "link":link_list,
                "dt":date_list,
                "dt_time":time_list
             }
        )

        # connect to local db
        with pyodbc.connect(db_statement) as conn:
            cursor = conn.cursor()
            try:
                for row in range(len(df)):
                    # build a tuple to be used to add to the df
                    line =tuple([df.iloc[row,0],df.iloc[row,1],df.iloc[row,2],df.iloc[row,3],df.iloc[row,4]])

                    #execute query
                    cursor.execute(sql, line)
            except Exception as e:
                print("An Error Ocurred:",e)
                #stop if error
                raise
            #commit insert if not error on building df
            conn.commit()
            print("Update Performed on ",datetime.datetime.now())

    else:
        print("An Error Ocurred:",e)
            
        #run cells above
        #display(Javascript('IPython.notebook.execute_cells_above()'))
    
    print("Next refresh on 3 hrs")
    
    print(df.head())
    
    # Make code stop for 1800 s before running again
    time.sleep(10800)

Update Performed on  2021-02-20 13:26:37.622624
Next refresh on 3 hrs
                                             caption  \
0   Gilberto desabafa com Karol Conká e a sister ...   
1   BBB 21: Mimado, Projota chantageia produção p...   
2   Na xepa, Projota se irrita: 'Se eu passar fom...   
3   BBB 21: Até a Thaís já ultrapassou Karol Conk...   
4   Sisters voltam da compra da xepa e Thaís avis...   

                                               image  \
0  https://conteudo.imguol.com.br/c/entreteniment...   
1  https://conteudo.imguol.com.br/c/parceiros/74/...   
2  https://conteudo.imguol.com.br/c/entreteniment...   
3  https://conteudo.imguol.com.br/c/entreteniment...   
4  https://conteudo.imguol.com.br/c/entreteniment...   

                                                link          dt dt_time  
0  https://tvefamosos.uol.com.br/noticias/redacao...  20/02/2021   13h15  
1  https://www.uol.com.br/splash/colunas/chico-ba...  20/02/2021   13h11  
2  https://tvefamosos.uol.com.b

In [ ]:
del df